In [1]:
import os
import numpy as np
import pandas as pd
from utils.utils import *
from utils.kabsch2D import *
from nsd_access import NSDAccess 

In [2]:
nsda = NSDAccess(nsd_dir)

In [5]:
def create_rotation_df(subj, rois):
    cols = ['source', 'target', 'U', 'error']
    rotations = np.zeros(((len(rois) * (len(rois)-1)) * 2, len(cols)), dtype=object)
    i = 0 
    for roi_source in rois.keys():
        mds_source = np.load(f'/home/stan/thesis-repo/data/MDS/betas_split/{subj}_37_{roi_source}_mds_betas_train.npy', allow_pickle=False)
        for roi_target in rois.keys():
            if roi_source != roi_target:
                for j in range(2):
                    mds_target = np.load(f'/home/stan/thesis-repo/data/MDS/betas_split/{subj}_37_{roi_target}_mds_betas_train.npy', allow_pickle=False)
                    # flip the target when j is at 1, does nothing when j = 0 
                    if j == 1:
                        mds_target = np.dot(mds_target, np.array([[-1, 0], [0, 1]]))
                        roi_target = roi_target + "_flipped"
                    U, t= kabsch2D(mds_source, mds_target, translate=True )
                    rotations[i, 0] = roi_source
                    rotations[i, 1] = roi_target
                    rotations[i, 2] = U
                    rotated_source = rotate(mds_source, U)
                    rotations[i, 3] = error(rotated_source, mds_target, t)
                    i += 1
    return pd.DataFrame(rotations, columns=cols)

In [7]:
rotations_df = create_rotation_df(subj='subj01', rois=rois)
rotations_df

,source,target,U,error
0,V1,V2,"[[0.9937364, 0.111750215], [-0.111750215, 0.99...",5843.815918
1,V1,V2_flipped,"[[0.2857228217385375, 0.9583123025077827], [-0...",4923.302201
2,V1,V3,"[[0.25467932, 0.9670255], [-0.9670255, 0.25467...",6156.311523
3,V1,V3_flipped,"[[0.9206654035364477, 0.39035267993324924], [-...",5277.029346
4,V1,hV4,"[[-0.8642162, -0.50312066], [0.50312066, -0.86...",6783.592773
...,...,...,...,...
259,TO-2,LO-1_flipped,"[[0.740268298892458, 0.6723115688837029], [-0....",7223.579939
260,TO-2,LO-2,"[[-0.026073754, -0.99966], [0.99966, -0.026073...",6625.286621
261,TO-2,LO-2_flipped,"[[0.6885233656526, 0.7252141579908771], [-0.72...",5522.83092
262,TO-2,TO-1,"[[-0.6986541, -0.7154597], [0.7154597, -0.6986...",4708.388184


In [8]:
rotations_df.groupby('source')['error'].mean().sort_values()

source
LO-2     6009.395812
VO-1     6264.381235
PHC-1    6620.142321
LO-1     6644.516198
PHC-2    6669.469479
TO-1     6779.044393
VO-2     6825.655275
V3       6929.276943
TO-2     6956.757963
hV4      7070.416379
V2       7083.409528
V1       7408.058236
Name: error, dtype: object

In [ ]:
c